## Aim
#### Context
We plan to create a strkfarm managed ETH/wstETH strategy on Ekubo. We need data analysis of past data to suggest LP parameters to manage investment with max profit
End usage: We will be providing certain liquidity to this pool in the suggested range and we will also rebalance the position from time to time to maintain maximum fee output while keeping impermanent loss less
#### Methodology
Divide total available liquidity and swaps data into two parts: 70-30. 70% being used for training and optimisation, 30% used for testing the result
#### Expected output
Analyse the liquidity and swaps data of ETH/wstETH pool with best TVL
Suggest logic for configuring lower & upper ticks for adding liquidity along with conditions to trigger rebalance
Show strategy performance on the test data
_

⚠️ Note: Overfitting must be avoided. Do whatever optimisations u want on training data, once u run test on test data, share the result as it is. Do not optimise on test data.

## Detailed methodology
1. Install starknet.py to read events from ekubo using !pip install starknet.py (https://starknetpy.readthedocs.io/en/latest/account_creation.html)
2. Key concepts of ekubo: https://docs.ekubo.org/about-ekubo/key-concepts
3. Ekubo protocol, PositionUpdated and swap event's contract addresses are found through this. https://app.ekubo.org/charts/wstETH/ETH
4. Block number in which ekubo protocol has been created is found by looking at the "Ekubo positions NFT" the token address of this nft is obtained by looking at the transfer event preceding or succeeding the add liquidity event https://voyager.online/event/669266_158_2
https://voyager.online/tx/0x14cd53e2a6836e77da41706d4c6f0d07749c561acc3c52cd344fcc1ff920316
4. The value of pool fee and tick spacing is taken for the pool with highest TVL https://app.ekubo.org/charts/wstETH/ETH
5. To identify the notation in which pool fee and tick spacing are written refer to https://docs.ekubo.org/integration-guides/reference/reading-pool-price
6. Filtered events are then divided into 70% for training and 30% for testing. 
7. The training sample is used for simulating investment for different liquidity ranges and rebalance ranges. To identify the right range for liquidity and rebalancing. 
8. a fixed liquidity is taken for now and the fee is not computed as percentage. Once the initial testing is done and the formulae is verified, will make further changes. 


In [2]:
# !pip install starknet.py

# Python standard imports
import json
import math
import pickle
import time

# External library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from statistics import median
from starknet_py.contract import Contract
from starknet_py.net.account.account import Account
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.models.chains import StarknetChainId
from starknet_py.net.signer.stark_curve_signer import KeyPair
import yfinance as yf

# Internal imports

In [3]:
# Ekubo contract address
contract_address = '0x00000005dd3d2f4429af886cd1a3b08289dbcea99a294197e9eb43b0e0325b4b' # ekubo
event_key_position_updated = ['0x3a7adca3546c213ce791fabf3b04090c163e419c808c9830fb343a4a395946e'] # for PositionUpdated event
event_key_swap = ['0x157717768aca88da4ac4279765f09f4d0151823d573537fbbeb950cdbd9a870'] #for swapped event
from_block= 655177 #165388 # beginning of ekubo protocol  
to_block=671277#"latest" # 669044
# Creates an instance of account which is already deployed
# Account using transaction version = 1 (has __validate__ function)
# client = FullNodeClient(node_url="https://starknet-mainnet.infura.io/v3/9c1398098092415980ad945d193fe32d")
client = FullNodeClient(node_url="https://starknet-mainnet.public.blastapi.io")

In [ ]:
# # Pool keys
# pool_fee = 0.001 # in %
# tick_spacing = 0.002 # in %

# pool_fee_touse = 3402823669209384797702963685910118
# print(int(pool_fee_touse))
# tick_spacing_touse = math.ceil(math.log(1+tick_spacing/100) / math.log(1.000001))
# print(tick_spacing_touse)

In [35]:
# start_block = 615403
# block_chunk_size = 10000

# # This is the search condition
# def search_condition(event):
#     data = event.data
#     try:
#         # Check for the first condition
#         if (hex(data[2]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7") and \
#            (hex(data[1]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2") and \
#            (data[3] == pool_fee_touse) and \
#            (data[4] == tick_spacing_touse):
#             return True
        
#         # Check for the invalid order condition
#         elif (hex(data[1]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7") and \
#              (hex(data[2]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2"):
#             raise Exception("invalid order")
    
#     except IndexError as e:
#         print(f"Error while checking event data: {e}")
    
#     return False

# while True:
#     # Define the range for the block search
#     from_block = start_block
#     to_block = start_block + block_chunk_size - 1

#     try:
#         # Fetch the events within the current block range
#         events_response_positions_updated = await client.get_events(
#             address=contract_address,
#             keys=[event_key_position_updated],  # for PositionUpdated event
#             from_block_number=from_block,
#             to_block_number=to_block,
#             follow_continuation_token=True,
#             chunk_size=47,
#         )
        
#         # Check if any event satisfies the condition
#         found_event = False
#         for event_no, event in enumerate(events_response_positions_updated.events):
#             if search_condition(event):  # Apply the condition to each event
#                 print(f"Event satisfying the condition found at block {event.block_number} (Event No: {event_no})")
#                 found_event = True
#                 break
        
#         if found_event:
#             break  # Exit the loop if an event was found
        
#         # If no event was found, move to the next chunk of blocks
#         start_block += block_chunk_size
#         print(f"No event found in blocks {from_block} to {to_block}, searching next range...")

#     except Exception as e:
#         print(f"An error occurred while fetching events: {e}")
#         break  # Exit the loop if an error occurs


3402823669209384797702963685910118
20
No event found in blocks 615403 to 625402, searching next range...
No event found in blocks 625403 to 635402, searching next range...
No event found in blocks 635403 to 645402, searching next range...
Event satisfying the condition found at block 655177 (Event No: 10801)


In [37]:
# from_block= 165388 # beginning of ekubo protocol  
# to_block= "latest" # 669044
# event_key_pool_initialized = "0x25ccf80ee62b2ca9b97c76ccea317c7f450fd6efb6ed6ea56da21d7bb9da5f1"
# events_response_pool_initialized = await client.get_events(
#     address = contract_address,
#     keys = [[event_key_pool_initialized]], # for swap event
#     from_block_number = from_block,
#     to_block_number = to_block,
#     follow_continuation_token=True,
#     chunk_size=47,
# )
# print("len", len(events_response_pool_initialized.events))

# # This is the search condition extracted from your logic
# def search_condition(event):
#     data = event.data
#     try:
#         # Check for the first condition
#         if (hex(data[2]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7") and \
#            (hex(data[1]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2") and \
#            (data[3] == pool_fee_touse) and \
#            (data[4] == tick_spacing_touse):
#             print("found", event)
#             return True
        
#         # Check for the invalid order condition
#         elif (hex(data[1]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7") and \
#              (hex(data[2]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2"):
#             raise Exception("invalid order")
    
#     except IndexError as e:
#         print(f"Error while checking event data: {e}")
    
#     return False

#  # Check if any event satisfies the condition
# found_event = False
# for event_no, event in enumerate(events_response_positions_updated.events):
#     if search_condition(event):  # Apply the condition to each event
#         print(f"Event satisfying the condition found at block {event.block_number} (Event No: {event_no})")
#         found_event = True
#         break

len 565
found EmittedEvent(from_address=158098919692956613592021320609952044916245725306097615271255138786123, keys=[1653201459205670538619725765890364513302126541856407219635220342091629827182], data=[1301608360075471435988652873011016112527098238822505666695223302941012672615, 1886212889629631188189497155848883534738756148921111726686756987927630157522, 2087021424722619777119509474943472645767659996348769578120564519014510906823, 3402823669209384797702963685910118, 20, 0, 634500, 158280, 0, 158460, 0, 27408128235124552785321, 0, 0, 0, 2670000000000000000, 0], transaction_hash=2879145539418839403171763982604352401380370371521015479181609416093330785856, block_hash=1007256769720868385100059786946035728971940593564070340067263501679755643489, block_number=655177)
Event satisfying the condition found at block 655177 (Event No: 10801)


In [39]:
# events_response_positions_updated = await client.get_events(
#     address = contract_address,
#     keys = [event_key_position_updated], # for PositionUpdated event
#     from_block_number = from_block,
#     to_block_number = to_block,
#     follow_continuation_tokaen=True,
#     chunk_size=47,
# )
# print(len(events_response_positions_updated.events))


# # Save the entire events_response_positions_updated object to a file
# with open('events_response_positions_updated.pkl', 'wb') as f:
#     pickle.dump(events_response_positions_updated, f)

# print("Events data saved to events_response_positions_updated.pkl")

25192


In [4]:
# Load the events_response_positions_updated object from a file
with open('events_response_positions_updated.pkl', 'rb') as f:
    events_response_positions_updated = pickle.load(f)

print("Events data loaded successfully")
print((events_response_positions_updated.events[1]))

Events data loaded successfully
EmittedEvent(from_address=158098919692956613592021320609952044916245725306097615271255138786123, keys=[1653201459205670538619725765890364513302126541856407219635220342091629827182], data=[1301608360075471435988652873011016112527098238822505666695223302941012672615, 2087021424722619777119509474943472645767659996348769578120564519014510906823, 2368576823837625528275935341135881659748932889268308403712618244410713532584, 0, 1001, 0, 0, 88722634, 1, 88722634, 0, 0, 0, 0, 0, 0, 0], transaction_hash=3380792800644086123602274049313571131378918353183444763251013603515006644394, block_hash=1007256769720868385100059786946035728971940593564070340067263501679755643489, block_number=655177)


In [4]:
# max_retries = 50
# retry_delay = 5  # seconds
# attempts = 0
# total_retrieved_events = 0  # Accumulator to track the total number of events retrieved

# events_response_swap = None

# while attempts < max_retries:
#     try:
#         events_response_swap = await client.get_events(
#             address=contract_address,
#             keys=[event_key_swap],  # for swap event
#             from_block_number=from_block,
#             to_block_number=to_block,
#             follow_continuation_token=True,
#             chunk_size=47,
#         )
        
#         # Update total retrieved events
#         events_retrieved = len(events_response_swap.events)
#         total_retrieved_events += events_retrieved
        
#         # Print how many events were retrieved this time and the total so far
#         print(f"Successfully retrieved {events_retrieved} events in this attempt")
#         print(f"Total events retrieved so far: {total_retrieved_events}")
        
#         break  # Exit loop after successful retrieval
#     except Exception as e:
#         attempts += 1
#         print(f"Attempt {attempts} failed with error: {e}")
#         if attempts < max_retries:
#             print(f"Retrying in {retry_delay} seconds...")
#             time.sleep(retry_delay)
#         else:
#             print("Max retries reached. Exiting.")
#             raise
            
# # Save the entire events_response_positions_updated object to a file
# with open('events_response_swap.pkl', 'wb') as f:
#     pickle.dump(events_response_swap, f)

# print("Events data saved to events_response_swap.pkl")

Attempt 1 failed with error: 
Retrying in 5 seconds...
Attempt 2 failed with error: 
Retrying in 5 seconds...
Attempt 3 failed with error: Cannot connect to host starknet-mainnet.public.blastapi.io:443 ssl:default [getaddrinfo failed]
Retrying in 5 seconds...
Attempt 4 failed with error: Cannot connect to host starknet-mainnet.public.blastapi.io:443 ssl:default [getaddrinfo failed]
Retrying in 5 seconds...
Attempt 5 failed with error: Cannot connect to host starknet-mainnet.public.blastapi.io:443 ssl:default [getaddrinfo failed]
Retrying in 5 seconds...
Attempt 6 failed with error: Cannot connect to host starknet-mainnet.public.blastapi.io:443 ssl:default [getaddrinfo failed]
Retrying in 5 seconds...
Attempt 7 failed with error: Cannot connect to host starknet-mainnet.public.blastapi.io:443 ssl:default [getaddrinfo failed]
Retrying in 5 seconds...
Attempt 8 failed with error: Cannot connect to host starknet-mainnet.public.blastapi.io:443 ssl:default [getaddrinfo failed]
Retrying in 5 s

In [ ]:
# # Configuration variables
# max_retries = 50
# retry_delay = 5  # seconds
# attempts = 0
# total_retrieved_events = 0  # Accumulator to track the total number of events retrieved
# chunk_size = 1000  # Number of blocks to retrieve per iteration

# events_response_swap = None
# current_from_block = from_block
# combined_events2 = []  # List to store all retrieved events

# while current_from_block <= to_block:
#     attempts = 0
#     while attempts < max_retries:
#         try:
#             # Retrieve events for the current chunk of blocks
#             events_response_swap = await client.get_events(
#                 address=contract_address,
#                 keys=[event_key_swap],  # for swap event
#                 from_block_number=current_from_block,
#                 to_block_number=min(current_from_block + chunk_size - 1, to_block),
#                 follow_continuation_token=True,
#                 chunk_size=47,  # Adjust chunk_size if necessary for API constraints
#             )
            
#             # Update total retrieved events
#             events_retrieved = len(events_response_swap.events)
#             total_retrieved_events += events_retrieved
            
#             # Print how many events were retrieved this time and the total so far
#             print(f"Successfully retrieved {events_retrieved} events for blocks {current_from_block} to {min(current_from_block + chunk_size - 1, to_block)}")
#             print(f"Total events retrieved so far: {total_retrieved_events}")
            
#             # Append the retrieved events to the combined list
#             combined_events2.extend(events_response_swap)
            
#             # Update the block range for the next iteration
#             current_from_block += chunk_size
            
#             # Exit the retry loop since this attempt was successful
#             break
            
#         except Exception as e:
#             attempts += 1
#             print(f"Attempt {attempts} failed with error: {e}")
#             if attempts < max_retries:
#                 print(f"Retrying in {retry_delay} seconds...")
#                 time.sleep(retry_delay)
#             else:
#                 print("Max retries reached. Exiting.")
#                 raise

# # Save the combined events to a single file after processing all blocks
# final_filename = 'combined_events_response_swap2.pkl'
# with open(final_filename, 'wb') as f:
#     pickle.dump(combined_events2, f)
# print(f"Combined events data saved to {final_filename}")

In [5]:
# Load the events_response_swap object from a file
with open('combined_events_response_swap.pkl', 'rb') as f:
    combined_events_response_swap = pickle.load(f)
print("Events data loaded successfully")

print(len(combined_events_response_swap))

Events data loaded successfully
1738080


In [49]:
# # Load the events_response_swap object from a file
# with open('combined_events_response_swap.pkl', 'rb') as f:
#     combined_events_response_swap = pickle.load(f)
# print("Events data loaded successfully")

# swap_events_ekubo_df =  pd.DataFrame(combined_events_response_swap)
# swap_events_ekubo_df['keys'] = swap_events_ekubo_df['keys'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

# data_column_names_swap = ['locker', 'token0', 'token1', 'fee', 'tickspacing', 'extension', 'amount', 'amount_bool', 'is_token1', 'sqrt_ratio_limit', 
#      'sqrt_ratio_limit_bool', 'skip_ahead', 'amount0','amount0_bool', 'amount1', 'amount1_bool', 'sqrt_ratio_after',
#     'sqrt_ratio_after_bool', 'tick_after', 'tick_after_bool', 'liquidity_after']

    
# # Assuming df is your DataFrame and 'data' is the column with arrays of 17 elements
# data_split = pd.DataFrame(swap_events_ekubo_df['data'].tolist(), columns=data_column_names_swap)

# # Concatenating the new columns back to the original DataFrame
# swap_events_ekubo_df = pd.concat([swap_events_ekubo_df, data_split], axis=1)

# # Dropping the original 'data' column if you don't need it anymore
# swap_events_ekubo_df = swap_events_ekubo_df.drop(columns=['data'])

# # Display the modified DataFrame
# print(swap_events_ekubo_df.head())

# # filename = f'swap_events_ekubo_from_{from_block}_to_{to_block}.parquet'


# # Save DataFrame as Feather file
# swap_events_ekubo_df.to_feather(f'swap_events_ekubo_from_{from_block}_to_{to_block}.feather')


# # # Convert all columns with large integers to float
# # for col in events_response_swap_df.columns:
# #     if pd.api.types.is_integer_dtype(events_response_swap_df[col]):
# #         # Convert to float if any value exceeds 64-bit integer limit
# #         if events_response_swap_df[col].max() > (2**63 - 1):
# #             events_response_swap_df[col] = events_response_swap_df[col].astype(str)

# # # Save the DataFrame as a Parquet file
# # swap_events_ekubo_df.to_parquet(filename, engine='pyarrow', index=False)

# # print("Data saved successfully in Parquet format")

Events data loaded successfully
                                        from_address  \
0  1580989196929566135920213206099520449162457253...   
1  1580989196929566135920213206099520449162457253...   
2  1580989196929566135920213206099520449162457253...   
3  1580989196929566135920213206099520449162457253...   
4  1580989196929566135920213206099520449162457253...   

                                                keys  \
0  6068116598133324338764937091720144931722643757...   
1  6068116598133324338764937091720144931722643757...   
2  6068116598133324338764937091720144931722643757...   
3  6068116598133324338764937091720144931722643757...   
4  6068116598133324338764937091720144931722643757...   

                                    transaction_hash  \
0  5379951863121538683099002174430011598502365039...   
1  5379951863121538683099002174430011598502365039...   
2  5379951863121538683099002174430011598502365039...   
3  4855896733655157058425204165595320457715756741...   
4  48558967336

In [9]:
# existing values from the website 
pool_fee = 0.001  # in %
tick_spacing = 0.002  # in %

# Constants to use
pool_fee_touse = 3402823669209384797702963685910118
tick_spacing_touse = math.ceil(math.log(1 + tick_spacing / 100) / math.log(1.000001))

In [10]:
# To store valid events
valid_events_positions = []
valid_events_swap = []

# Processing positions updated events
for event_no in range(len(events_response_positions_updated.events)):
    event_data = events_response_positions_updated.events[event_no].data
    
    # Check for the first condition
    if (hex(event_data[2]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7" and \
        hex(event_data[1]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2" and \
        event_data[3] == pool_fee_touse and \
        event_data[4] == tick_spacing_touse):
        valid_events_positions.append(event_data)  # Save the valid event data
    
    # Check for invalid order condition
    elif (hex(event_data[1]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7" and \
          hex(event_data[2]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2"):
        
        raise Exception("invalid order")

# Processing swap events
for event_no_swap in range(len(combined_events_response_swap)):
    event_data = combined_events_response_swap[event_no_swap].data
    
    # Check for the first condition
    if (hex(event_data[2]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7" and \
        hex(event_data[1]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2" and \
        event_data[3] == pool_fee_touse and \
        event_data[4] == tick_spacing_touse):
        valid_events_swap.append(event_data)  # Save the valid event data
    
    # Check for invalid order condition
    elif (hex(event_data[1]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7" and \
          hex(event_data[2]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2"):
        raise Exception("invalid order")

In [11]:
print(len(valid_events_positions))
print(len(valid_events_swap))

576
20248


In [12]:
def bool_to_sign(x):
    if x == 0:
        return 1
    elif x == 1:
        return -1
    else:
        raise Exception("invalid input")

ekubo_eth_weth_liquidity_data = []
ekubo_eth_weth_swap_data = []

for event_no in range(len(events_response_positions_updated.events)):
    event_data = {
        'block_number': events_response_positions_updated.events[event_no].block_number,
        'transaction_hash': events_response_positions_updated.events[event_no].transaction_hash,  
        'delta_liquidity': events_response_positions_updated.events[event_no].data[11]*bool_to_sign(events_response_positions_updated.events[event_no].data[12]),
        'tick_lower': events_response_positions_updated.events[event_no].data[7]*bool_to_sign(events_response_positions_updated.events[event_no].data[8]),
        'tick_upper': events_response_positions_updated.events[event_no].data[9]*bool_to_sign(events_response_positions_updated.events[event_no].data[10])
    }
        # Check for the first condition
    if (hex(events_response_positions_updated.events[event_no].data[2]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7" and \
        hex(events_response_positions_updated.events[event_no].data[1]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2" and \
        events_response_positions_updated.events[event_no].data[3] == pool_fee_touse and \
        events_response_positions_updated.events[event_no].data[4] == tick_spacing_touse):
            ekubo_eth_weth_liquidity_data.append(event_data)

        
for event_no_swap in range(len(combined_events_response_swap)):
    event_data = {
        'block_number': combined_events_response_swap[event_no_swap].block_number,
        'transaction_hash': combined_events_response_swap[event_no_swap].transaction_hash,
        'price': 1.000001**(-combined_events_response_swap[event_no_swap].data[18] * bool_to_sign(combined_events_response_swap[event_no].data[19])),
        'tick_id': -combined_events_response_swap[event_no_swap].data[18] * bool_to_sign(combined_events_response_swap[event_no].data[19]),
        'liquidity_after': combined_events_response_swap[event_no_swap].data[20],
        'amount0': combined_events_response_swap[event_no_swap].data[12] * bool_to_sign(combined_events_response_swap[event_no].data[13]), # 0 swapping +ve 
        'amount1': combined_events_response_swap[event_no_swap].data[14] * bool_to_sign(combined_events_response_swap[event_no].data[15])
    }
    if (hex(combined_events_response_swap[event_no_swap].data[2]) == "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7" and \
        hex(combined_events_response_swap[event_no_swap].data[1]) == "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2" and \
        combined_events_response_swap[event_no_swap].data[3] == pool_fee_touse and \
        combined_events_response_swap[event_no_swap].data[4] == tick_spacing_touse):
            ekubo_eth_weth_swap_data.append(event_data)
    


In [13]:
# Create a pandas DataFrame
liquidity_df = pd.DataFrame(ekubo_eth_weth_liquidity_data)
print(min(liquidity_df.tick_lower))
print(min(liquidity_df.tick_upper))

swap_df = pd.DataFrame(ekubo_eth_weth_swap_data)
print(max(swap_df.tick_id))
print(min(swap_df.tick_id))


swap_df['fee_token1_ETH'] = (pool_fee/100 * swap_df['amount1'].abs()) / 10**18

print(max(swap_df.fee_token1_ETH))

# # Convert to Parquet using pyarrow
# table = pa.Table.from_pandas(df)
# pq.write_table(table, 'events_data.parquet')

# print("Data saved to Parquet file successfully!")

123100
158440
162969
158443
0.01


In [14]:
# Step 1: Collect block numbers and prices from ekubo_eth_weth_swap_data
block_price_data = [{'block_number': event['block_number'], 'price': event['price']} for event in ekubo_eth_weth_swap_data]

# Step 2: Convert the collected data into a pandas DataFrame
df = pd.DataFrame(block_price_data)

# Step 3: Group by block number and calculate the median price for each block
median_prices_df = df.groupby('block_number')['price'].median().reset_index()
median_prices_df.columns = ['block_number', 'median_price']

# Step 4: Define the functions to calculate and round tick_id from prices
def price_to_tick_id(price):
    return int(np.log(price) / np.log(1.000001))

def round_to_tick_spacing(tick_id, tick_spacing_touse):
    return round(tick_id / tick_spacing_touse) * tick_spacing_touse

price_range_liquidity = 20
# Step 5: Add ±5% price boundaries and convert them to tick_id
median_prices_df['price_minus_20%'] = median_prices_df['median_price'] * (1 - (price_range_liquidity*0.01))
median_prices_df['price_plus_20%'] = median_prices_df['median_price'] * (1 + (price_range_liquidity*0.01))
median_prices_df['tick_id_median'] = median_prices_df['median_price'].apply(price_to_tick_id)
median_prices_df['tick_id_minus_20%'] = median_prices_df['price_minus_20%'].apply(price_to_tick_id)
median_prices_df['tick_id_plus_20%'] = median_prices_df['price_plus_20%'].apply(price_to_tick_id)

# Step 6: Round the tick_ids to the nearest multiple of tick_spacing_touse
tick_spacing_touse = 20  # Define tick_spacing
median_prices_df['tick_id_minus_20%_rounded'] = median_prices_df['tick_id_minus_20%'].apply(lambda x: round_to_tick_spacing(x, tick_spacing_touse))
median_prices_df['tick_id_plus_20%_rounded'] = median_prices_df['tick_id_plus_20%'].apply(lambda x: round_to_tick_spacing(x, tick_spacing_touse))

# Step 7: Create an array of tick IDs for each block
median_prices_df['tick_id_array'] = median_prices_df.apply(
    lambda row: np.arange(row['tick_id_minus_20%_rounded'], row['tick_id_plus_20%_rounded'] + tick_spacing_touse, tick_spacing_touse), axis=1
)

# Step 8: Retain the necessary columns
final_df = median_prices_df[['block_number', 'median_price', 'tick_id_median', 'tick_id_array']]

# Step 9: Distribute cumulative liquidity per block
cumulative_liquidity_per_tick = {}

liquidity_per_block = []  # To store cumulative liquidity for each block

for idx, row in final_df.iterrows():
    block_number = row['block_number']
    tick_array = row['tick_id_array']

    # Initialize cumulative liquidity if not already initialized
    if not cumulative_liquidity_per_tick:
        cumulative_liquidity_per_tick = {tick: 0 for tick in tick_array}

    # Clone the current cumulative state so that we can update it for the current block
    current_block_liquidity = cumulative_liquidity_per_tick.copy()

    # Get liquidity data for the current block only
    liquidity_data_for_block = [event for event in ekubo_eth_weth_liquidity_data if event['block_number'] == block_number]
    
    # Add the current block's liquidity to the cumulative state
    for liquidity_event in liquidity_data_for_block:
        tick_lower = liquidity_event['tick_lower']
        tick_upper = liquidity_event['tick_upper']
        delta_liquidity = liquidity_event['delta_liquidity']
        
        # Distribute liquidity to the relevant ticks in the cumulative state
        for tick in tick_array:
            if tick_lower <= tick <= tick_upper:
                current_block_liquidity[tick] += delta_liquidity
    
    # Update the cumulative liquidity state
    cumulative_liquidity_per_tick = current_block_liquidity.copy()

    # Store the cumulative liquidity distribution for the current block
    liquidity_per_block.append({
        'block_number': block_number,
        'tick_liquidity_distribution': current_block_liquidity
    })

# # Step 10: Create a DataFrame for cumulative liquidity distribution
# liquidity_df = pd.DataFrame(liquidity_per_block)

# # Step 11: Print all non-zero cumulative liquidity ticks and their values
# for idx, row in liquidity_df.iterrows():
#     block_number = row['block_number']
#     tick_liquidity_distribution = row['tick_liquidity_distribution']
    
#     # Filter out ticks with zero liquidity
#     non_zero_liquidity = {tick: liquidity for tick, liquidity in tick_liquidity_distribution.items() if liquidity != 0}
    
#     # Print non-zero liquidity ticks
#     print(f"Block Number: {block_number}")
#     if non_zero_liquidity:
#         for tick, liquidity in non_zero_liquidity.items():
#             print(f"  Tick: {tick}, Liquidity: {liquidity}")
#     print("-" * 40)

# # Step 12: Plot the cumulative liquidity distribution for the last block
# last_block_data = liquidity_df.iloc[-1]
# last_block_number = last_block_data['block_number']
# last_block_tick_liquidity = last_block_data['tick_liquidity_distribution']

# # Convert the tick liquidity distribution for plotting
# ticks = list(last_block_tick_liquidity.keys())
# liquidity = list(last_block_tick_liquidity.values())

# # Plot the last block's cumulative tick liquidity distribution
# plt.figure(figsize=(10, 6))
# plt.bar(ticks, liquidity, color='green')
# plt.xlabel('Tick')
# plt.ylabel('Cumulative Liquidity')
# plt.title(f'Cumulative Liquidity Distribution Across Ticks for Block {last_block_number}')
# plt.xticks(rotation=90)
# plt.show()


In [15]:
# Step 1: Merge liquidity_df and final_df on block_number
liquidity_distribution_df = pd.merge(liquidity_df, final_df, on='block_number', how='inner')

# Step 2: Inspect the combined DataFrame
print(liquidity_distribution_df.head())


   block_number                                   transaction_hash  \
0        655177  2879145539418839403171763982604352401380370371...   
1        655424  1279209847316592012798496238689033441847594858...   
2        655424  2236106133222411803388378305320859355758706418...   
3        655424  3387282097104070895993181893437770629177012930...   
4        655424  3779414541303248423459591667090630130681763650...   

            delta_liquidity  tick_lower  tick_upper  median_price  \
0   27408128235124552785321      158280      158460      1.171710   
1  -99209107691145427163307      158400      158440      1.171794   
2     262573697381870119149      158000      158800      1.171794   
3   10990777850625690682059      158220      158860      1.171794   
4   99205850788549541510734      158540      158580      1.171794   

   tick_id_median                                      tick_id_array  
0          158463  [-64680.0, -64660.0, -64640.0, -64620.0, -6460...  
1          158536  [-6

In [17]:
# Assuming both swap_df and liquidity_distribution_df are already created
# and contain the 'block_number' column

# Calculate the maximum, minimum, and split block number
max_block = max(swap_df['block_number'])
min_block = min(swap_df['block_number'])
split_block = round(min_block + (max_block - min_block) * 0.7)

print("Split Block:", split_block)
print("Max Block:", max_block)
print("Min Block:", min_block)

# Split the data into the first 70% and the remaining 30% for both DataFrames
# For swap_df
train_swap_df = swap_df[swap_df['block_number'] <= split_block]
test_swap_df = swap_df[swap_df['block_number'] > split_block]

# For liquidity_distribution_df
train_liquidity_df = liquidity_distribution_df[liquidity_distribution_df['block_number'] <= split_block]
test_liquidity_df = liquidity_distribution_df[liquidity_distribution_df['block_number'] > split_block]

# Inspect the resulting DataFrames
print("Training Swap DataFrame:")
print(train_swap_df.head())
print("\nTesting Swap DataFrame:")
print(test_swap_df.head())

print("\nTraining Liquidity DataFrame:")
print(train_liquidity_df.head())
print("\nTesting Liquidity DataFrame:")
print(test_liquidity_df.head())


Split Block: 666447
Max Block: 671277
Min Block: 655177
Training Swap DataFrame:
   block_number                                   transaction_hash     price  \
0        655177  2416157948790894051047941157778594406573451511...  1.171710   
1        655177  2416157948790894051047941157778594406573451511...  1.171710   
2        655179  1241404949126819184019688044659591190675013926...  1.171708   
3        655179  1241404949126819184019688044659591190675013926...  1.171708   
4        655179  1241404949126819184019688044659591190675013926...  1.171708   

   tick_id           liquidity_after            amount0             amount1  \
0   158464  137703958037274544831372   2256616757779677   -2644126307134438   
1   158464  137703958037274544831372                  0                   0   
2   158463  137703958037274544831372  42668433630093793  -49994504035756152   
3   158463  137703958037274544831372  21336138963323289  -25000000000000000   
4   158463  137703958037274544831372  21334

In [ ]:
# optimize to identify range in which I should invest and when to book impermanent loss
# Triggers to think about 
# 1. Price crosses outside of the current range: Move the range closer to the new price.
# 2. Increased volatility: Widen the range to accommodate more price movements.
# 3. Liquidity depletion: Reallocate liquidity if the current position is largely consumed.
# 4. Fee accumulation: Rebalance after reaching a threshold of fees.
# 5. Time-based rebalance: Rebalance periodically to optimize positions.

In [18]:
# Function to convert price to tick
def price_to_tick(price):
    return int(math.log(price) / math.log(1.0001))

# Function to calculate impermanent loss
def impermanent_loss(price_ratio):
    return 1 - math.sqrt((4 * price_ratio) / ((price_ratio + 1) ** 2))

initial_liquidity = 10000000
fee_percentage = 0.00001
# Function to run the simulation for a given liquidity and rebalancing range
def run_simulation(liquidity_range, rebalance_range):
    initial_price = train_swap_df['price'].iloc[0]  # Initial price from the data
    fees_earned = 0
    impermanent_loss_total = 0
    current_liquidity = initial_liquidity
    
    for swap_event in range(len(train_swap_df)):
        current_price = train_swap_df['price'].iloc[swap_event]
        amount_swapped = train_swap_df['amount1'].iloc[swap_event]
        available_liquidity = train_swap_df['liquidity_after'].iloc[swap_event]
        
        current_tick = price_to_tick(current_price)
        
        # Calculate the bounds of the liquidity range (ticks)
        liquidity_lower_bound = initial_price * (1 - liquidity_range)
        liquidity_upper_bound = initial_price * (1 + liquidity_range)

        tick_lower_bound = price_to_tick(liquidity_lower_bound)
        tick_upper_bound = price_to_tick(liquidity_upper_bound)
        
        # Check if the current tick is within the range of added liquidity
        if tick_lower_bound <= current_tick <= tick_upper_bound:
            # Each tick has liquidity equal to the current liquidity
            fees_earned += abs(amount_swapped) / 10**18 * fee_percentage * current_liquidity / available_liquidity
            
        # Check if the current price is within the rebalancing range
        rebalance_lower_bound = initial_price * (1 - rebalance_range)
        rebalance_upper_bound = initial_price * (1 + rebalance_range)
        
        if not (rebalance_lower_bound <= current_price <= rebalance_upper_bound):
            # Price is out of rebalance range, rebalance
            price_ratio = current_price / initial_price
            il = impermanent_loss(price_ratio)
            impermanent_loss_total += il
            
            # Update liquidity position by adding earned fees
            current_liquidity += fees_earned
            fees_earned = 0  # Reset fees after rebalancing
            
            # Reset initial price to the new price after rebalancing
            initial_price = current_price
            print('i')
    
    # Final calculation of returns
    net_earnings = fees_earned - impermanent_loss_total
    
    return net_earnings, fees_earned, impermanent_loss_total

# Run the simulation and generate the optimization results
optimization_results = []
liquidity_range_options = np.arange(0.001, 0.021, 0.005)  # Liquidity add range from 5% to 20%
rebalance_range_options = np.arange(0.001, 0.021, 0.005)  # Rebalance range from 5% to 20%

for liquidity_range in liquidity_range_options:
    for rebalance_range in rebalance_range_options:
        net_earnings, fees_earned, impermanent_loss_total = run_simulation(liquidity_range, rebalance_range)
        optimization_results.append({
            "Liquidity Range (%)": liquidity_range * 100,
            "Rebalance Range (%)": rebalance_range * 100,
            "Net Earnings (ETH)": net_earnings,
            "Total Fees Earned (ETH)": fees_earned,
            "Impermanent Loss (ETH)": impermanent_loss_total
        })

# Convert optimization results to a DataFrame for analysis
optimization_df = pd.DataFrame(optimization_results)

# Sort by net earnings to find the best performing combination
optimization_df_sorted = optimization_df.sort_values(by="Net Earnings (ETH)", ascending=False)

# Display the top 5 performing combinations
print(optimization_df_sorted.head())


i
i
i
i
i
i
i
i
    Liquidity Range (%)  Rebalance Range (%)  Net Earnings (ETH)  \
5                   0.6                  0.6        3.284067e-19   
6                   0.6                  1.1        3.284067e-19   
7                   0.6                  1.6        3.284067e-19   
9                   1.1                  0.6        3.284067e-19   
10                  1.1                  1.1        3.284067e-19   

    Total Fees Earned (ETH)  Impermanent Loss (ETH)  
5              3.284067e-19                     0.0  
6              3.284067e-19                     0.0  
7              3.284067e-19                     0.0  
9              3.284067e-19                     0.0  
10             3.284067e-19                     0.0  


In [ ]:
# Verify prices

# Define the ticker symbols for ETH and wETH
eth_symbol = 'ETH-USD'
weth_symbol = 'WETH-USD'

# Fetch historical data for both tokens over a given period (e.g., 1 month)
eth_data = yf.Ticker(eth_symbol).history(period="1mo")
weth_data = yf.Ticker(weth_symbol).history(period="1mo")

# Align both dataframes by the same date index (merge them on the 'Date' column)
data = pd.DataFrame({
    'ETH Price': eth_data['Close'],
    'wETH Price': weth_data['Close']
})

# Drop any rows with missing data (if there are any mismatched dates)
data.dropna(inplace=True)

# Calculate the ratio of ETH price to wETH price for each date
data['Price Ratio (ETH/wETH)'] = data['ETH Price'] / data['wETH Price']

# Determine the entire price ratio range
min_ratio = data['Price Ratio (ETH/wETH)'].min()
max_ratio = data['Price Ratio (ETH/wETH)'].max()

# Print the results
print(f"Minimum ETH/wETH Price Ratio: {min_ratio}")
print(f"Maximum ETH/wETH Price Ratio: {max_ratio}")

# Optional: Plot the price ratios over time
data['Price Ratio (ETH/wETH)'].plot(title="ETH/wETH Price Ratio Over Time")
plt.xlabel('Date')
plt.ylabel('Price Ratio')
plt.show()


In [ ]:
# ETH: "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7" 
# wstETH: "0x42b8f0484674ca266ac5d08e4ac6a3fe65bd3129795def2dca5c34ecc5f96d2"
# price = 1.0001^(tick_id or tick_value)
    
    
    {'locker', 'token0', 'token1', 'fee', 'tickspacing', 'extension', 'salt', 'lower', 'lower_bool', 'upper', 
     'upper_bool', 'delta_liquidity', 'delta_liquidity_bool', 'amount0','amount0_bool', 'amount1', 'amount1_bool'}
# For idetifying the index of 
Position_updated
EmittedEvent(
    from_address=158098919692956613592021320609952044916245725306097615271255138786123, 
    keys=[1653201459205670538619725765890364513302126541856407219635220342091629827182], 
    data=[locker:1301608360075471435988652873011016112527098238822505666695223302941012672615,  #0
          poolkey: token0:2087021424722619777119509474943472645767659996348769578120564519014510906823,  #1
          token1:2368576823837625528275935341135881659748932889268308403712618244410713532584,  #2
          fee:170141183460469235273462165868118016,  #3
          tickspacing:1000,  #4
          Extension:0,  #5
          Salt: 238207,  #6
          lower: 88722000, 1,  #7, 8
          Upper: 88722000, 0,  #9, 10 
          Delta_liquidity: 24934382157, 1,  #11, 12
          Amount0: 433198613657447, 1,  #13, 14
          Amount1: 1433756, 1],  #15, 16
      transaction_hash=879945867416280128092979369806709446749761821654825519062415897008764242951,
      block_hash=2421102280480996809432903330067643765909910418548957016097347656801334531512,
      block_number=663452)



    {'locker', 'token0', 'token1', 'fee', 'tickspacing', 'extension', 'amount', 'amount_bool', 'is_token1', 'sqrt_ratio_limit', 
     'sqrt_ratio_limit_bool', 'skip_ahead', 'amount0','amount0_bool', 'amount1', 'amount1_bool', 'sqrt_ratio_after',
    'sqrt_ratio_after_bool', 'tick_after', 'tick_after_bool', 'liquidity_after'}

Swap
EmittedEvent(
    from_address=158098919692956613592021320609952044916245725306097615271255138786123, 
    keys=[606811659813332433876493709172014493172264375712319669538606486437773289584], 
    data=[Locker: 1951224604049798639815629089492514906598247071757934301055136774321924456550, #0
          Poolkey: token0: 2087021424722619777119509474943472645767659996348769578120564519014510906823, #1
          token1: 2368576823837625528275935341135881659748932889268308403712618244410713532584, #2
          fee: 170141183460469235273462165868118016,  #3
          tick_spacing 1000,  #4
          Extension: 0,  #5
          
          Params: Amount (mag, sign) 18750000000000000, 0,  #6, 7
          isToken1: 0,  #8
          sqrt_ratio_limit: 15411102256202172767744790565534148, 0,  #9, 10
          skip_ahead: 0,  #11
          
          Amount0: 18750000000000000, 0,  #12,13
          Amount1: 49541253, 1, #14, 15
          sqrt_Ratio_After: 17495541587887167702017650483399816, 0, # 16, 17
          tick_after: 19751180, 1, #18, 19
          liquidity_after: 59517410719957310], #20
    transaction_hash=1028460612028350658488036399834335161397563811459669946359742869978934007182,
    block_hash=2750810716156707596732812189537740646505960575694968173768545834980232403465, 
    block_number=668044)


In [13]:
hex(1028460612028350658488036399834335161397563811459669946359742869978934007182)

'0x246168c475c5ca4a9204306e73470f652f433abf5a55a6d847ea0083625d58e'